In [2]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Optuna
import optuna
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO


# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False
from sklearn.metrics import r2_score

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt
import seaborn as sn

# Advertencias
import warnings
warnings.filterwarnings("ignore")

In [3]:
stations = [1, 2, 3, 5, 6, 7, 8, 9, 10]

validacion = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates=['FECHAHORA'])

lista = []

for i in stations:

    df_aux = validacion[validacion['ESTACION'] == i]

    first_pred = df_aux.FECHAHORA.min() + relativedelta(days = 2)

    df_aux = df_aux[df_aux['FECHAHORA'] >= first_pred]

    lista.append(df_aux)

df = pd.concat(lista)
df.reset_index(inplace = True, drop = True)

pred_xgb = pd.read_csv('datos/experimento_full/prueba_predicciones_todaslasestaciones_validation_xgboost_5_1_2.csv')
pred_xgb.reset_index(inplace = True, drop = True)


In [4]:
pred_xgb.rename(columns = {'FORECAST': 'FORECAST_XGB'}, inplace = True)

p_xgb = pred_xgb.FORECAST_XGB

df = pd.merge(p_xgb, df, right_index = True, left_index = True)

In [5]:
score = r2_score(df.AQI_MP2_5, df.FORECAST_XGB)

print(score)

0.7437502207007188


In [4]:
for i in stations:
    
    fig_val = go.Figure()

    df_grafica = df[df['ESTACION'] == i]

    fig_val.add_trace(
        go.Scatter( y = list(df_grafica.FORECAST_XGB), x = list(df_grafica.FECHAHORA), name = 'Forecasts XGB', line=dict(color = 'Orange')))

    fig_val.add_trace(
        go.Scatter( y = list(df_grafica.AQI_MP2_5), x = list(df_grafica.FECHAHORA), name = 'Target', line=dict(color = '#0087e8')))

    layout = go.Layout(
            title="Title",
            xaxis=dict(
                title="X Label"
            ),
            yaxis=dict(
                title="Y label"
            ) ) 

    fig_val.update_layout( 
                            width = 600,
                            height = 700,
                            title = {
                                    'text': "12hr prediction. XGBOOST. Station " + str(i),
                                    'y':0.9,
                                    'x':0.5,
                                    'xanchor': 'center',
                                    'yanchor': 'top'}, 
                            
                            yaxis_title = 'AQI MP 2.5', 
                            xaxis_title = 'Date',
                            
                            legend= {
                                    'xanchor': 'center',
                                    'yanchor': 'bottom',
                                    'y' : -0.7,
                                    'x' : 0.0,
                            },
                            font=dict(
                                        size=16,
                                            ),
                            legend_title = 'References',
                            )
    
    fig_val.write_html('graphs/modelos_finales_todas_estaciones/val_24h_station_'+str(i)+'.html')
